In [1]:
import json
import os
import numpy as np
import pprint
import pandas as pd
import re

from IPython.core.display import display
from pymongo import MongoClient
from gender_detector.gender_detector import GenderDetector

# Create Age Features

In [2]:
# -*- coding: utf-8 -*-
def is_english(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True
    

def first_name(s):
    return s.split()[0]


def last_name(s):
    return s.split()[-1]


def clean_name(name):
    '''Utility function to clean names by removing links, special characters using simple regex statements.'''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", name).split())

In [3]:
for f in os.listdir('../data/raw'):

    df = pd.read_csv('../data/raw/' + f)

    df['user_first_name'] = df.user_name.apply(first_name)
    df['user_last_name'] = df.user_name.apply(last_name)
    df['user_name_is_english'] = df.user_first_name.apply(is_english)

    #df.user_first_name[df['user_name_is_english'] == False]

    df_clean = df[df['user_name_is_english']].copy()

    detector = GenderDetector('us')
    df_clean['user_gender'] =  df_clean.user_first_name.apply(detector.guess)

    df_clean['gender_male'] = df_clean.user_gender == 'male'
    df_clean['gender_female'] = df_clean.user_gender == 'female'
    df_clean['gender_unknown'] = df_clean.user_gender == 'unknown'
    
    break

# Create Age Features

In [4]:
df = df_clean.copy()

In [35]:
from agefromname import AgeFromName
from agefromname import GenerationFromName

In [36]:
name = df.user_first_name[3]
gender = df.user_gender[3]
print(name)
print(gender)

Pax
male


In [46]:
age_from_name = AgeFromName()
age = 2017 - age_from_name.argmax(name, 'm', 2017, minimum_age=20)
print(age)

43


In [49]:
generation_from_name = GenerationFromName()
generation_probs = generation_from_name.get_estimated_counts(
    name, 'm', 2017)
display(generation_probs)

Baby Boomers    0.000000e+00
Generation X    1.115362e+01
Generation Z    9.822559e+01
Greatest        0.000000e+00
Millenials      0.000000e+00
Post Gen Z      2.422186e+02
Silent          0.000000e+00
_other         -5.684342e-14
Name: estimated_count, dtype: float64

In [50]:
type(generation_probs)

pandas.core.series.Series